# Word2Vec in TensorFlow
Below is a carbon copy of the [basic word2vec example][2] from a [TensorFlow word tutorial][3].

In [1]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Basic word2vec example."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

# Give a folder path as an argument with '--log_dir' to save
# TensorBoard summaries. Default is a log folder in current directory.
current_path = os.path.dirname(os.path.realpath(sys.argv[0]))

parser = argparse.ArgumentParser()
parser.add_argument(
    '--log_dir',
    type=str,
    default=os.path.join(current_path, 'log'),
    help='The log directory for TensorBoard summaries.')
FLAGS, unparsed = parser.parse_known_args()

# Create the directory for TensorBoard variables if there is not.
if not os.path.exists(FLAGS.log_dir):
  os.makedirs(FLAGS.log_dir)

# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'


# pylint: disable=redefined-outer-name
def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  local_filename = os.path.join(gettempdir(), filename)
  if not os.path.exists(local_filename):
    local_filename, _ = urllib.request.urlretrieve(url + filename,
                                                   local_filename)
  statinfo = os.stat(local_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception('Failed to verify ' + local_filename +
                    '. Can you get to it with a browser?')
  return local_filename


filename = maybe_download('text8.zip', 31344016)


# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data


vocabulary = read_data(filename)
print('Data size', len(vocabulary))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000


def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    index = dictionary.get(word, 0)
    if index == 0:  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary


# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(
    vocabulary, vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0


# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)  # pylint: disable=redefined-builtin
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    if data_index == len(data):
      buffer.extend(data[0:span])
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels


batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0],
        reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()

with graph.as_default():

  # Input data.
  with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    with tf.name_scope('embeddings'):
      embeddings = tf.Variable(
          tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
      embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
      nce_weights = tf.Variable(
          tf.truncated_normal(
              [vocabulary_size, embedding_size],
              stddev=1.0 / math.sqrt(embedding_size)))
    with tf.name_scope('biases'):
      nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  with tf.name_scope('loss'):
    loss = tf.reduce_mean(
        tf.nn.nce_loss(
            weights=nce_weights,
            biases=nce_biases,
            labels=train_labels,
            inputs=embed,
            num_sampled=num_sampled,
            num_classes=vocabulary_size))

  # Add the loss value as a scalar to summary.
  tf.summary.scalar('loss', loss)

  # Construct the SGD optimizer using a learning rate of 1.0.
  with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
                                            valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Merge all summaries.
  merged = tf.summary.merge_all()

  # Add variable initializer.
  init = tf.global_variables_initializer()

  # Create a saver.
  saver = tf.train.Saver()

# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # Open a writer to write summaries.
  writer = tf.summary.FileWriter(FLAGS.log_dir, session.graph)

  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size, num_skips,
                                                skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # Define metadata variable.
    run_metadata = tf.RunMetadata()

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    # Also, evaluate the merged op to get all summaries from the returned "summary" variable.
    # Feed metadata variable to session for visualizing the graph in TensorBoard.
    _, summary, loss_val = session.run(
        [optimizer, merged, loss],
        feed_dict=feed_dict,
        run_metadata=run_metadata)
    average_loss += loss_val

    # Add returned summaries to writer in each step.
    writer.add_summary(summary, step)
    # Add metadata to visualize the graph for the last run.
    if step == (num_steps - 1):
      writer.add_run_metadata(run_metadata, 'step%d' % step)

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

  # Write corresponding labels for the embeddings.
  with open(FLAGS.log_dir + '/metadata.tsv', 'w') as f:
    for i in xrange(vocabulary_size):
      f.write(reverse_dictionary[i] + '\n')

  # Save the model for checkpoints.
  saver.save(session, os.path.join(FLAGS.log_dir, 'model.ckpt'))

  # Create a configuration for visualizing embeddings with the labels in TensorBoard.
  config = projector.ProjectorConfig()
  embedding_conf = config.embeddings.add()
  embedding_conf.tensor_name = embeddings.name
  embedding_conf.metadata_path = os.path.join(FLAGS.log_dir, 'metadata.tsv')
  projector.visualize_embeddings(writer, config)

writer.close()

# Step 6: Visualize the embeddings.


# pylint: disable=missing-docstring
# Function to draw visualization of distance between embeddings.
def plot_with_labels(low_dim_embs, labels, filename):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(
        label,
        xy=(x, y),
        xytext=(5, 2),
        textcoords='offset points',
        ha='right',
        va='bottom')

  plt.savefig(filename)


try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(
      perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels, os.path.join(gettempdir(), 'tsne.png'))

except ImportError as ex:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')
  print(ex)

Found and verified text8.zip
Data size 17005207
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
3084 originated -> 5239 anarchism
3084 originated -> 12 as
12 as -> 6 a
12 as -> 3084 originated
6 a -> 195 term
6 a -> 12 as
195 term -> 2 of
195 term -> 6 a
Initialized
Average loss at step  0 :  276.875
Nearest to such: cytotoxic, jay, transaction, vegeta, lay, vigilance, hermeticism, mailed,
Nearest to can: fillmore, amplified, inductees, dictators, demeaning, financi, compatriots, marriages,
Nearest to however: arrangement, instead, indentation, favourably, purport, revere, neutral, dell,
Nearest to often: acetate, ana, neoliberalism, gladly, zaragoza, afrobeat, cues, favorite,
Nearest to there: downturn, totleben, organisers, maneuvered, radha, watched, pulse, napol,
Nearest to

Average loss at step  46000 :  5.265518395423889
Average loss at step  48000 :  5.21418430519104
Average loss at step  50000 :  4.990064399838448
Nearest to such: well, known, keto, senex, vigilance, arsenic, marek, rear,
Nearest to can: may, will, should, would, could, must, timurid, eight,
Nearest to however: but, arrangement, that, dell, analyze, buffalo, and, dasyprocta,
Nearest to often: usually, still, also, which, generally, it, aidan, ana,
Nearest to there: it, they, which, he, downturn, who, totleben, rhotic,
Nearest to between: with, in, from, apparel, three, five, seven, alif,
Nearest to of: in, for, circ, agouti, agave, coke, and, lauder,
Nearest to only: counterparts, depressions, judicial, operatorname, complications, utterance, arin, circ,
Nearest to during: in, at, on, while, stipulate, subkey, existing, following,
Nearest to have: had, has, are, be, were, armored, gigantopithecus, circ,
Nearest to other: constitutional, circ, eclipse, expelled, franchisees, two, are, b

In [3]:
with open('word2vec_basic.nonbin', 'wt') as f:
    f.write('{} {}\n'.format(vocabulary_size, embedding_size))
    for index, embedding in enumerate(final_embeddings):
        f.write('{} {}\n'.format(reverse_dictionary[index], ' '.join(str(feature) for feature in embedding)))

## Evaluation
The below code uses the [kudkudak/word-embeddings-benchmarks package][1] to evaluate the above embeddings on the [Google Analogy test set][2].

 [1]: https://github.com/kudkudak/word-embeddings-benchmarks (Package for evaluating word embeddings)
 [2]: https://aclweb.org/aclwiki/Google_analogy_test_set_(State_of_the_art) (Google analogy test set)

In [4]:
from annoy import AnnoyIndex

n_trees = 100
search_k = 100

annoy_index = AnnoyIndex(embedding_size, metric='angular')
for index, embedding in enumerate(final_embeddings):
    annoy_index.add_item(index, embedding)
annoy_index.build(n_trees)

True

In [9]:
from web.datasets.analogy import fetch_google_analogy

dataset = fetch_google_analogy()

In [10]:
from __future__ import division

previous_category = dataset.category[0]
trials = {'semantic': [], 'syntactic': [], 'total': []}
missing_words = set()
skipped_trials = 0

def print_acc():
    print('- Semantic accuracy:\t{} %\n- Syntactic accuracy:\t{} %\n- Total accuracy:\t{} %'.format(
        float('nan') if not trials['semantic'] else sum(trials['semantic']) / len(trials['semantic']) * 100,
        float('nan') if not trials['syntactic'] else sum(trials['syntactic']) / len(trials['syntactic']) * 100,
        float('nan') if not trials['total'] else sum(trials['total']) / len(trials['total']) * 100,
    ))
    print('- Missing words:\t{} (skipped {} % of all trials)'.format(
        ', '.join(missing_words),
        float('nan') if not trials['total'] else skipped_trials / len(trials['total']) * 100,
    ))
    print()

for X, y, category, category_high_level in zip(
            dataset.X,
            dataset.y,
            dataset.category,
            dataset.category_high_level
        ):

    if previous_category != category:
        previous_category = category
        print('Current category: {}'.format(category))
        print_acc()

    if X[0] in dictionary and X[1] in dictionary and X[2] in dictionary and y in dictionary:
        u1 = final_embeddings[dictionary[X[0]]]
        v1 = final_embeddings[dictionary[X[1]]]
        u2 = final_embeddings[dictionary[X[2]]]
        v2_index = annoy_index.get_nns_by_vector(u2 + (v1 - u1), 1, search_k)[0]
        v2_word = reverse_dictionary[v2_index]
        print('{} - {} + {} = {} (correct answer: {})'.format(X[1], X[0], X[2], v2_word, y))
        trial_success = v2_word == y
    else:
        if X[0] not in dictionary:
            missing_words.add(X[0])
        if X[1] not in dictionary:
            missing_words.add(X[1])
        if X[2] not in dictionary:
            missing_words.add(X[2])
        if y not in dictionary:
            missing_words.add(y)
        skipped_trials += 1
        trial_success = False
    trials[category_high_level].append(trial_success)
    trials['total'].append(trial_success)

print('Finished!')
print_acc()

greece - athens + baghdad = hadadezer (correct answer: iraq)
greece - athens + bangkok = electrons (correct answer: thailand)
greece - athens + beijing = cryptography (correct answer: china)
greece - athens + berlin = greece (correct answer: germany)
greece - athens + bern = arrigo (correct answer: switzerland)
greece - athens + cairo = possibility (correct answer: egypt)
greece - athens + canberra = archers (correct answer: australia)
greece - athens + hanoi = superior (correct answer: vietnam)
greece - athens + havana = meighen (correct answer: cuba)
greece - athens + helsinki = helsinki (correct answer: finland)
greece - athens + islamabad = greece (correct answer: pakistan)
greece - athens + kabul = max (correct answer: afghanistan)
greece - athens + london = greece (correct answer: england)
greece - athens + madrid = engraver (correct answer: spain)
greece - athens + moscow = moscow (correct answer: russia)
greece - athens + oslo = bombarded (correct answer: norway)
greece - athen

egypt - cairo + caracas = caracas (correct answer: venezuela)
egypt - cairo + chisinau = modulated (correct answer: moldova)
egypt - cairo + conakry = institutes (correct answer: guinea)
egypt - cairo + copenhagen = egypt (correct answer: denmark)
egypt - cairo + dakar = dakar (correct answer: senegal)
egypt - cairo + damascus = egypt (correct answer: syria)
egypt - cairo + dhaka = batcave (correct answer: bangladesh)
egypt - cairo + doha = abroad (correct answer: qatar)
egypt - cairo + dublin = moor (correct answer: ireland)
egypt - cairo + gaborone = belgrade (correct answer: botswana)
egypt - cairo + georgetown = liz (correct answer: guyana)
egypt - cairo + hanoi = timescales (correct answer: vietnam)
egypt - cairo + havana = lrv (correct answer: cuba)
egypt - cairo + helsinki = va (correct answer: finland)
egypt - cairo + islamabad = endowment (correct answer: pakistan)
egypt - cairo + jakarta = jakarta (correct answer: indonesia)
egypt - cairo + kabul = preaching (correct answer: 

nicaragua - managua + stockholm = wrinkled (correct answer: sweden)
nicaragua - managua + suva = visa (correct answer: fiji)
nicaragua - managua + taipei = beneath (correct answer: taiwan)
nicaragua - managua + tallinn = nicaragua (correct answer: estonia)
nicaragua - managua + tashkent = nicaragua (correct answer: uzbekistan)
nicaragua - managua + tbilisi = buccaneer (correct answer: georgia)
nicaragua - managua + tegucigalpa = subgenius (correct answer: honduras)
nicaragua - managua + tehran = nicaragua (correct answer: iran)
nicaragua - managua + tokyo = nicaragua (correct answer: japan)
bahrain - manama + manila = tubular (correct answer: philippines)
bahrain - manama + minsk = bahrain (correct answer: belarus)
bahrain - manama + mogadishu = constituted (correct answer: somalia)
bahrain - manama + monrovia = odds (correct answer: liberia)
bahrain - manama + montevideo = libre (correct answer: uruguay)
bahrain - manama + moscow = champlain (correct answer: russia)
bahrain - manama +

lithuania - vilnius + chisinau = align (correct answer: moldova)
lithuania - vilnius + conakry = screws (correct answer: guinea)
lithuania - vilnius + copenhagen = chromaticity (correct answer: denmark)
lithuania - vilnius + dakar = morse (correct answer: senegal)
poland - warsaw + yerevan = poland (correct answer: armenia)
poland - warsaw + zagreb = wrestler (correct answer: croatia)
poland - warsaw + accra = verrocchio (correct answer: ghana)
poland - warsaw + algiers = poland (correct answer: algeria)
poland - warsaw + amman = amman (correct answer: jordan)
poland - warsaw + ankara = oversized (correct answer: turkey)
poland - warsaw + asmara = exuberant (correct answer: eritrea)
poland - warsaw + astana = discredited (correct answer: kazakhstan)
poland - warsaw + athens = throughput (correct answer: greece)
poland - warsaw + baghdad = orange (correct answer: iraq)
poland - warsaw + baku = hemiparesis (correct answer: azerbaijan)
poland - warsaw + bamako = pleasing (correct answer: 

colorado - denver + lexington = detractors (correct answer: kentucky)
colorado - denver + stockton = unicameral (correct answer: california)
colorado - denver + cincinnati = brigitte (correct answer: ohio)
colorado - denver + anchorage = ym (correct answer: alaska)
colorado - denver + toledo = fishes (correct answer: ohio)
colorado - denver + plano = plano (correct answer: texas)
colorado - denver + henderson = shires (correct answer: nevada)
colorado - denver + orlando = adrenaline (correct answer: florida)
colorado - denver + laredo = thanksgiving (correct answer: texas)
colorado - denver + chandler = colorado (correct answer: arizona)
colorado - denver + madison = augustine (correct answer: wisconsin)
colorado - denver + lubbock = collins (correct answer: texas)
colorado - denver + garland = bantam (correct answer: texas)
colorado - denver + glendale = enchanted (correct answer: arizona)
colorado - denver + reno = reno (correct answer: nevada)
maryland - baltimore + nashville = staf

...

french - france + china = french (correct answer: chinese)
french - france + colombia = folkloric (correct answer: colombian)
french - france + croatia = sifra (correct answer: croatian)
french - france + denmark = english (correct answer: danish)
french - france + egypt = tibet (correct answer: egyptian)
german - germany + greece = prolog (correct answer: greek)
german - germany + iceland = deserving (correct answer: icelandic)
german - germany + india = tuxedo (correct answer: indian)
german - germany + ireland = ireland (correct answer: irish)
german - germany + israel = wired (correct answer: israeli)
german - germany + italy = ci (correct answer: italian)
german - germany + japan = japan (correct answer: japanese)
german - germany + korea = ffffff (correct answer: korean)
german - germany + macedonia = dancer (correct answer: macedonian)
german - germany + malta = reads (correct answer: maltese)
german - germany + mexico = dore (correct answer: mexican)
german - germany + moldova 

spanish - spain + albania = habsburg (correct answer: albanian)
spanish - spain + argentina = porter (correct answer: argentinean)
spanish - spain + australia = hydrophilic (correct answer: australian)
spanish - spain + austria = quack (correct answer: austrian)
spanish - spain + brazil = quack (correct answer: brazilian)
spanish - spain + bulgaria = bulgaria (correct answer: bulgarian)
spanish - spain + cambodia = spanish (correct answer: cambodian)
spanish - spain + chile = spanish (correct answer: chilean)
spanish - spain + china = broadcasters (correct answer: chinese)
spanish - spain + colombia = spanish (correct answer: colombian)
spanish - spain + croatia = spanish (correct answer: croatian)
spanish - spain + denmark = spanish (correct answer: danish)
spanish - spain + egypt = glorified (correct answer: egyptian)
spanish - spain + england = stella (correct answer: english)
spanish - spain + france = buzz (correct answer: french)
spanish - spain + germany = pavement (correct answ

paid - paying + playing = arcadius (correct answer: played)
paid - paying + predicting = movimiento (correct answer: predicted)
paid - paying + reading = destroys (correct answer: read)
paid - paying + running = running (correct answer: ran)
paid - paying + saying = viridian (correct answer: said)
paid - paying + seeing = player (correct answer: saw)
paid - paying + selling = bs (correct answer: sold)
paid - paying + shrinking = rural (correct answer: shrank)
paid - paying + singing = paid (correct answer: sang)
paid - paying + sitting = champlain (correct answer: sat)
paid - paying + sleeping = sleeping (correct answer: slept)
paid - paying + slowing = paid (correct answer: slowed)
paid - paying + spending = paid (correct answer: spent)
paid - paying + striking = feel (correct answer: struck)
paid - paying + swimming = tended (correct answer: swam)
paid - paying + taking = interviewing (correct answer: took)
paid - paying + thinking = execution (correct answer: thought)
paid - paying 

cats - cat + dream = budo (correct answer: dreams)
cats - cat + eagle = cats (correct answer: eagles)
cats - cat + elephant = tasmania (correct answer: elephants)
cats - cat + eye = fundamentalism (correct answer: eyes)
cats - cat + finger = cats (correct answer: fingers)
cats - cat + goat = goat (correct answer: goats)
cats - cat + hand = hand (correct answer: hands)
cats - cat + horse = horse (correct answer: horses)
cats - cat + lion = bordering (correct answer: lions)
cats - cat + machine = westmoreland (correct answer: machines)
cats - cat + man = cats (correct answer: men)
cats - cat + monkey = bmx (correct answer: monkeys)
cats - cat + mouse = provokes (correct answer: mice)
cats - cat + onion = awkward (correct answer: onions)
cats - cat + pear = cats (correct answer: pears)
cats - cat + pig = budo (correct answer: pigs)
cats - cat + pineapple = chadian (correct answer: pineapples)
cats - cat + rat = rat (correct answer: rats)
cats - cat + road = conveyor (correct answer: roads

estimates - estimate + talk = expertise (correct answer: talks)
estimates - estimate + think = sizing (correct answer: thinks)
estimates - estimate + vanish = kodak (correct answer: vanishes)
estimates - estimate + walk = septimania (correct answer: walks)
estimates - estimate + work = greenland (correct answer: works)
estimates - estimate + write = raum (correct answer: writes)
estimates - estimate + decrease = klerk (correct answer: decreases)
estimates - estimate + describe = erasing (correct answer: describes)
estimates - estimate + eat = ward (correct answer: eats)
estimates - estimate + enhance = pronunciation (correct answer: enhances)
finds - find + generate = rollout (correct answer: generates)
finds - find + go = limelight (correct answer: goes)
finds - find + implement = ejective (correct answer: implements)
finds - find + increase = increase (correct answer: increases)
finds - find + listen = listen (correct answer: listens)
finds - find + play = seeks (correct answer: play